[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/evals/voyageai-embeddings-eval.ipynb)

[![View Article](https://img.shields.io/badge/View%20Article-blue)](https://www.mongodb.com/developer/products/atlas/choose-embedding-model-rag/?utm_campaign=devrel&utm_source=cross-post&utm_medium=organic_social&utm_content=https%3A%2F%2Fgithub.com%2Fmongodb-developer%2FGenAI-Showcase&utm_term=apoorva.joshi)

# RAG Series Part 1: How to choose the right embedding model for your RAG application

This notebook evaluates the [voyage-lite-02-instruct](https://docs.voyageai.com/embeddings/) model.


## Step 1: Install required libraries

- **datasets**: Python library to get access to datasets available on Hugging Face Hub
<p>
- **voyageai**: Python library to interact with Voyage AI APIs
<p>
- **sentence-transformers**: Framework for working with text and image embeddings
<p>
- **numpy**: Python library that provides tools to perform mathematical operations on arrays
<p>
- **pandas**: Python library for data analysis, exploration and manipulation
<p>
- **tdqm**: Python module to show a progress meter for loops


In [3]:
! pip install -qU datasets voyageai sentence-transformers numpy pandas tqdm

## Step 2: Setup pre-requisites

Set Voyage API key as environment variable, and initialize the Voyage AI client.

Steps to obtain a Voyage AI API Key can be found [here](https://docs.voyageai.com/docs/api-key-and-installation).


In [4]:
import os
import getpass
import voyageai

In [5]:
VOYAGE_API_KEY = getpass.getpass("Voyage API Key:")
voyage_client = voyageai.Client(api_key=VOYAGE_API_KEY)

Voyage API Key:········


## Step 3: Download the evaluation dataset

We will use MongoDB's [cosmopedia-wikihow-chunked](https://huggingface.co/datasets/MongoDB/cosmopedia-wikihow-chunked) dataset, which has chunked versions of WikiHow articles from the [Cosmopedia](https://huggingface.co/datasets/HuggingFaceTB/cosmopedia) dataset released by Hugging Face. The dataset is pretty large, so we will only grab the first 25k records for testing.


In [6]:
from datasets import load_dataset
import pandas as pd

# Use streaming=True to load the dataset without downloading it fully
data = load_dataset("MongoDB/cosmopedia-wikihow-chunked", split="train", streaming=True)
# Get first 25k records from the dataset
data_head = data.take(25000)
df = pd.DataFrame(data_head)

# Use this if you want the full dataset
# data = load_dataset("AIatMongoDB/cosmopedia-wikihow-chunked", split="train")
# df = pd.DataFrame(data)

## Step 4: Data analysis

Make sure the length of the dataset is what we expect (25k), preview the data, drop Nones etc.


In [7]:
# Ensuring length of dataset is what we expect i.e. 25k
len(df)

25000

In [8]:
# Previewing the contents of the data
df.head()

,doc_id,chunk_id,text_token_length,text
0,0,0,180,Title: How to Create and Maintain a Compost Pi...
1,0,1,141,**Step 2: Gather Materials**\nGather brown (ca...
2,0,2,182,_Key guideline:_ For every volume of green mat...
3,0,3,188,_Key tip:_ Chop large items like branches and ...
4,0,4,157,**Step 7: Maturation and Use**\nAfter 3-4 mont...


In [9]:
# Only keep records where the text field is not null
df = df[df["text"].notna()]

In [10]:
# Number of unique documents in the dataset
df.doc_id.nunique()

4335

## Step 5: Creating embeddings

Define the embedding function, and run a quick test.


In [11]:
from typing import List

In [12]:
def get_embeddings(
    docs: List[str], input_type: str, model: str = "voyage-lite-02-instruct"
) -> List[List[float]]:
    """
    Get embeddings using the Voyage AI API.

    Args:
        docs (List[str]): List of texts to embed
        input_type (str): Type of input to embed. Can be "document" or "query".
        model (str, optional): Model name. Defaults to "voyage-lite-02-instruct".

    Returns:
        List[List[float]]: Array of embedddings
    """
    response = voyage_client.embed(docs, model=model, input_type=input_type)
    return response.embeddings

In [13]:
# Generating a test embedding
test_voyageai_embed = get_embeddings([df.iloc[0]["text"]], "document")

In [14]:
# Sanity check to make sure embedding dimensions are as expected i.e. 1024
len(test_voyageai_embed[0])

1024

## Step 6: Evaluation


### Measuring embedding latency

Create a local vector store (list) of embeddings for the entire dataset.


In [15]:
from tqdm.auto import tqdm

In [16]:
texts = df["text"].tolist()

In [17]:
batch_size = 128

In [18]:
embeddings = []
# Generate embeddings in batches
for i in tqdm(range(0, len(texts), batch_size)):
    end = min(len(texts), i + batch_size)
    batch = texts[i:end]
    # Generate embeddings for current batch
    batch_embeddings = get_embeddings(batch, "document")
    # Add to the list of embeddings
    embeddings.extend(batch_embeddings)

  0%|          | 0/196 [00:00<?, ?it/s]

### Measuring retrieval quality

- Create embedding for the user query
<p>
- Get the top 5 most similar documents from the local vector store using cosine similarity as the similarity metric


In [19]:
import numpy as np
from sentence_transformers.util import cos_sim

In [20]:
# Converting embeddings list to a Numpy array- required to calculate cosine similarity
embeddings = np.asarray(embeddings)

In [21]:
def query(query: str, top_k: int = 3) -> None:
    """
    Query the local vector store for the top 3 most relevant documents.

    Args:
        query (str): User query
        top_k (int, optional): Number of documents to return. Defaults to 3.
    """
    # Generate embedding for the user query
    query_emb = np.asarray(get_embeddings([query], "query"))
    # Calculate cosine similarity
    scores = cos_sim(query_emb, embeddings)[0]
    # Get indices of the top k records
    idxs = np.argsort(-scores)[:top_k]

    print(f"Query: {query}")
    for idx in idxs:
        print(f"Score: {scores[idx]:.4f}")
        print(texts[idx])
        print("--------")

In [22]:
query("Give me some tips to improve my mental health.")

Query: Give me some tips to improve my mental health.
Score: 0.9247
*Explanation:* Regularly assess symptom improvement and reassess treatment efficacy. Modifications to existing plans might be required as circumstances change.

Recommendations:

* Keep track of mood fluctuations, thoughts, and behaviors using mobile apps, journals, or calendars.
* Share progress updates with therapists and other members of the support network.
* Discuss any challenges encountered while implementing recommended strategies.

Conclusion

Managing suicidal thoughts necessitates a multifaceted approach encompassing various methods and tools. By following this comprehensive guide, individuals grappling with these difficult emotions can access valuable resources, cultivate essential skills, and foster connections vital to overcoming adversity.
--------
Score: 0.9243
Key Tips:

* Learn to recognize early signs of stress and address them proactively.
* Share concerns with trusted friends, family members, or me

In [23]:
query("Give me some tips for writing good code.")

Query: Give me some tips for writing good code.
Score: 0.9177
Step 6: Improve Code Quality
Strive for clean, readable, maintainable code. Adopt consistent naming conventions, indentation styles, and formatting rules. Utilize version control systems like Git to track changes and collaborate effectively. Leverage linters and static analyzers to enforce style guides automatically. Document your work using comments and dedicated documentation tools. High-quality code facilitates collaboration, promotes longevity, and simplifies troubleshooting.

Step 7: Embrace Best Practices
Follow established best practices relevant to your chosen language and domain. Examples include Object-Oriented Design Principles, SOLID principles, Test-Driven Development (TDD), Dependency Injection, Asynchronous Programming, etc. While seemingly overwhelming initially, integrating them gradually enhances design patterns, scalability, and extensibility. Consult authoritative blogs, books, and articles to stay update

In [24]:
query("How to create a basic webpage?")

Query: How to create a basic webpage?
Score: 0.9276
Creating a simple webpage with HTML

Step 1: Understand What HTML Is
HTML (HyperText Markup Language) is the standard markup language used to create web pages. It provides the structure of a webpage, while CSS (Cascading Style Sheets) and JavaScript handle the presentation and functionality respectively.

Key Tips and Guidelines:

* Familiarize yourself with basic HTML tags such as `<html>`, `<head>`, `<body>`, `<h1>` through `<h6>`, `<p>`, `<a>`, `<img>`, `<div>`, and `<span>`.
* Always start an HTML document with a doctype declaration, which informs the browser about the version of HTML being used. For example: `<!DOCTYPE html>`.
--------
Score: 0.9244
Creating a webpage with basic HTML is an exciting endeavor that allows you to share information, showcase your creativity, or build the foundation for a more complex website. This comprehensive guide will walk you through each step required to create a simple yet effective static webp

In [25]:
query(
    "What are some environment-friendly practices I can incorporate in everyday life?"
)

Query: What are some environment-friendly practices I can incorporate in everyday life?
Score: 0.9214
### Step 5: Embrace Resource Conservation

Another critical aspect of Jawa culture revolves around conserving resources due to limited availability on their home planet. Implement sustainable practices such as reducing waste, reusing materials, and recycling whenever possible.

Key Tips
--------

* Monitor daily consumption habits and identify areas where reductions could occur (water usage, electricity, paper products, etc.).
* Repair damaged goods instead of purchasing replacements whenever feasible.
* Participate in local environmental initiatives focused on preserving natural resources.

Why It Matters
---------------

Adopting eco-friendly attitudes and actions contributes positively to both individual wellbeing and global sustainability efforts. Moreover, practicing conservation aligns closely with Jawa values, further deepening your connection to the character.

Conclusion
-----